In [1]:
%pip -q install -U pip
%pip -q install model-registry boto3 roboflow dotenv

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
codeflare-sdk 0.30.0 requires pydantic<2, but you have pydantic 2.11.10 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from model_registry import ModelRegistry
from model_registry.utils import S3Params
from model_registry.exceptions import StoreError
import os

/opt/app-root/lib64/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "DataSet" shadows an attribute in parent "Artifact"
  warnings.warn(


In [7]:
#get s3 variables 
AWS_S3_ENDPOINT = os.environ.get('AWS_S3_ENDPOINT')
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
AWS_S3_BUCKET = os.environ.get('AWS_S3_BUCKET')

In [6]:
# connect to model registry 
USER = "Maria Vittoria Passarello"
CLUSTER_DOMAIN = 'apps.cluster-2lxjg.2lxjg.sandbox2810.opentlc.com'

In [5]:
# Set up the model registry connection
model_registry_url = f"https://model-registry1-rest.{CLUSTER_DOMAIN}"
author_name = USER

print(model_registry_url)

registry = ModelRegistry(server_address=model_registry_url, port=443, author=author_name, is_secure=False)

https://model-registry1-rest.apps.cluster-2lxjg.2lxjg.sandbox2810.opentlc.com


In [1]:
registered_model_name = "smokeandfire"
version="0.0.4"
username = USER 

NameError: name 'USER' is not defined

In [27]:
import glob
files = glob.glob("runs/detect/train*/weights/best.onnx")

if not files:  
    print("Nessun modello trovato")
    model_path = None
else:
    latest = max(files, key=os.path.getmtime)
    model_path = latest
    print(f"Using model: {model_path}")

Nessun modello trovato


In [8]:
s3_upload_params = S3Params(
    bucket_name=os.environ.get('AWS_S3_BUCKET'),
    s3_prefix= f"smokeandfire/v{version}",
)

try:
    registered_model = registry.upload_artifact_and_register_model(
        name=registered_model_name,
        model_files_path=model_path,
        model_format_name="onnx",
        author=username,
        model_format_version="1",
        version=version,
        description="YOLO model fine tuned on images of smoke and fire",
        metadata={
            "accuracy": 0.55,
            "fraction": 0.1,
            "epoch": 1,
            "license": "apache-2.0"
        },
        upload_params=s3_upload_params
    )

    print(f"'{registered_model_name}' version '{version}' has been registered here: https://rhods-dashboard-redhat-ods-applications.{CLUSTER_DOMAIN}/modelRegistry/{username}-registry/registeredModels/1/versions/{registry.get_model_version(registered_model_name, version).id}/details")

except StoreError:
    rmver = registry.get_model_version(registered_model_name, version)
    print(f"Model and version already exists:\n{rmver}")

NameError: name 'version' is not defined